# Backtest of Avellaneda and Stoikov Market Making
* **Asset** : `ETHUSDT`
* **Start** : `2023-06-01 00:00:00` 
* **End** :  `2023-06-01 11:59:59`
* **Timeframe** : `1 minute`

In [1]:
from typing import Union

import time
import math
import random

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

random.seed(1)

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
#params
dt = '1min' # timestep

## Read the Trades data

In [3]:
trades = pd.read_csv("data/ETHUSDT-trades-2023-06-01.csv", usecols=['time','price','qty'])
trades['datetime'] = pd.to_datetime(trades['time'], utc=True, unit='ms')
trades = trades.drop(['time'], axis=1)
trades = trades.set_index("datetime")
trades = trades.sort_index()
#resample it to timestep
trades = trades['price'].resample(dt).ohlc()
trades.head()

,open,high,low,close
datetime,,,,
2023-06-01 00:00:00+00:00,1872.85,1875.01,1872.84,1874.02
2023-06-01 00:01:00+00:00,1874.03,1874.71,1873.63,1874.70
2023-06-01 00:02:00+00:00,1874.71,1875.42,1874.70,1875.06
2023-06-01 00:03:00+00:00,1875.06,1876.74,1874.83,1874.83
2023-06-01 00:04:00+00:00,1874.84,1875.56,1874.67,1874.99


## Read the Quotes data

Compute the mid price as 

$$\text{mid_price} = \frac{\text{ask_price} + \text{bid_price}}{2}$$

In [4]:
quotes = pd.read_csv("data/ETHUSDT-bookTicker-2023-06-01.csv", usecols=['transaction_time', 'best_bid_price', 'best_bid_qty', 'best_ask_price', 'best_ask_qty'])
quotes['datetime'] = pd.to_datetime(quotes['transaction_time'], utc=True, unit='ms')
quotes = quotes.drop(['transaction_time'], axis=1)
quotes = quotes.set_index("datetime")

quotes['mid_price'] = (quotes['best_ask_price']+quotes['best_bid_price'])/2
quotes = quotes.sort_index()
quotes.head()

,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty,mid_price
datetime,,,,,
2023-06-01 00:00:00.027000+00:00,1872.84,55.728,1872.85,24.164,1872.845
2023-06-01 00:00:00.027000+00:00,1872.84,43.728,1872.85,24.164,1872.845
2023-06-01 00:00:00.031000+00:00,1872.84,59.898,1872.85,24.164,1872.845
2023-06-01 00:00:00.042000+00:00,1872.84,59.898,1872.85,23.592,1872.845
2023-06-01 00:00:00.042000+00:00,1872.84,59.925,1872.85,23.592,1872.845


In [5]:
#resample the mid price to timestep
mid_price = quotes["mid_price"].resample(dt).last()
mid_price.head()

datetime
2023-06-01 00:00:00+00:00    1874.025
2023-06-01 00:01:00+00:00    1874.705
2023-06-01 00:02:00+00:00    1875.055
2023-06-01 00:03:00+00:00    1874.835
2023-06-01 00:04:00+00:00    1874.985
Freq: T, Name: mid_price, dtype: float64

In [6]:
trades['mid_price'] = mid_price
trades.head()

,open,high,low,close,mid_price
datetime,,,,,
2023-06-01 00:00:00+00:00,1872.85,1875.01,1872.84,1874.02,1874.025
2023-06-01 00:01:00+00:00,1874.03,1874.71,1873.63,1874.70,1874.705
2023-06-01 00:02:00+00:00,1874.71,1875.42,1874.70,1875.06,1875.055
2023-06-01 00:03:00+00:00,1875.06,1876.74,1874.83,1874.83,1874.835
2023-06-01 00:04:00+00:00,1874.84,1875.56,1874.67,1874.99,1874.985


In [7]:
class MarketMaking:
    """
    Create an instance of Avellaneda and Stoikov's 
    High Frequency Trading in Limit order book.
    """
    def __init__(self, X:pd.DataFrame, risk_aversion:float=0.1, 
                 liquidity:float=1.5, sigma:float=1, 
                 percent_fee:float=0.00009): 
        """
        :param X: (pd.DataFrame) a dataframe containing 
                  open, high, low, close and mid_price.
        :param risk_aversion: (float) "gamma"risk aversion 
                                against holding inventory 
                                for the dealer. Default 0.1 
                                low risk aversion.
        :param liquidity: (float) "kappa" order book 
                             liquidity. Default 1.5 according 
                             to Gopalakrishnan et. al. [2000]
        :param sigma: (float) constant volatility. Default 1.
        :param percent_fee: (float) the percentage commission 
                             fee for market makers. Default is 
                             0.09% for binance.
        
        """
        self.gamma = risk_aversion
        self.kappa = liquidity
        self.X = X
        self.sigma = sigma
        self.fees = percent_fee
        
        self.T = 1.0
        self.M = X.shape[0]
        self.dt = 1.0/self.M
    
        self.q = 0
        self.w = 0
        self.fee = 0
        self.total_fee = 0
        
        self.equity = [0]
        self.bid = [np.nan]
        self.ask = [np.nan]
        self.spread = [np.nan]
        self.inventory = [0]
        self.reserve_price =[np.nan]
        
    def run(self):
        """
        Run the backtest. 
        """
        for t in range(1, self.M): 
            S = self.X['mid_price'].iloc[t-1]
            high, low, close = self.X['high'].iloc[t], self.X['low'].iloc[t], self.X['close'].iloc[t] 
            #reservation price    
            r =  S - self.q * self.gamma * (self.sigma ** 2) * (self.T - t/self.M)   
            #optimal spread
            spread = self.gamma * (self.sigma ** 2) * (self.T - t/self.M) + (2/self.sigma) * math.log(1 + (self.gamma/self.kappa))
            #optimal bid ask placement
            bid = r - spread/2.# bid
            ask = r + spread/2.# ask
            if high >= ask:
                #sell limit order is filled by market order
                self.q = self.q - 1# position
                self.w = self.w + ask# wealth
                self.fee = ask*self.fees
            if low <= bid:
                #buy limit order if filled by market order
                self.q = self.q + 1# position
                self.w = self.w - bid# wealth
                self.fee = ask*self.fees
            self.equity.append((self.w + self.q * close) - self.fee)
            self.bid.append(bid)
            self.ask.append(ask)
            self.reserve_price.append(r)
            self.spread.append(spread)
            self.inventory.append(self.q)
            self.total_fee += self.fee
            self.fee = 0
            
    def results(self):
        """
        Display results of backtest.
        """
        print("                   Results              ")
        print("----------------------------------------")
        print("%14s %21s" % ('statistic', 'value'))
        print(40 * "-")
        print("%14s %20.2f" % ("Average spread :", np.nanmean(np.array(self.spread))))
        print("%16s %20.2f" % ("Profit :", self.equity[-1]))
        print("%16s %20.2f" % ("Std(Profit) :", np.array(self.equity).std()))
        print("%16s %20.0f" % ("End Inventory :", int(self.inventory[-1])))
        print("%16s %20.2f" % ("Total Fees :", self.total_fee))
        
        x = self.X.index

        # Create Bokeh figure objects
        fig1 = figure(title='Prices', width=800, height=400, x_axis_type='datetime')
        fig1.line(x, self.X['mid_price'], line_width=1., legend_label='Mid-market price', color='navy')
        fig1.line(x, self.reserve_price, line_width=1., legend_label='Indifference price', color='orange')
        fig1.circle(x, self.ask, size=1, line_color="red", fill_color="red", fill_alpha=0.8, legend_label='Priced ask')
        fig1.circle(x, self.bid, size=1, line_color="green", fill_color="green", fill_alpha=0.8, legend_label='Priced bid')
        fig1.grid.grid_line_alpha = 0.3
        fig1.legend.location = 'top_right'
        fig1.xaxis.axis_label = 'Time'
        fig1.yaxis.axis_label = 'Stock Price'
        
        pnl = np.diff(np.array(self.equity))
        

        fig2 = figure(title='Position', width=800, height=400, x_axis_type='datetime')
        fig2.line(x, self.inventory, line_color='green', line_width=1., legend_label='q')
        fig2.grid.grid_line_alpha = 0.3
        fig2.legend.location = 'top_right'
        fig2.xaxis.axis_label = 'Time'
        fig2.yaxis.axis_label = 'Position'
        
        
        fig3 = figure(title='Histogram', width=400, height=400)
        fig3.quad(top=np.histogram(pnl, bins=100)[0], bottom=0, left=np.histogram(pnl, bins=100)[1][:-1], right=np.histogram(pnl, bins=100)[1][1:], fill_color='red', line_color='black', legend_label='Inventory Strategy') 
        fig3.grid.grid_line_alpha = 0.3
        fig3.legend.location = 'top_right'
        fig3.xaxis.axis_label = 'pnl'
        fig3.yaxis.axis_label = 'N'

        fig4 = figure(title='Equity', width=400, height=400, x_axis_type='datetime')
        fig4.line(x, self.equity, line_width=1., legend_label='equity', color='navy')
        fig4.grid.grid_line_alpha = 0.3
        fig4.legend.location = 'top_right'
        fig4.xaxis.axis_label = 'Time'
        fig4.yaxis.axis_label = 'Price'

        # Create a grid layout of figures
        grid = gridplot([[fig3, fig4]])
        show(fig1)
        show(fig2)
        # Show the plot
        show(grid)

In [8]:
mm = MarketMaking(trades, risk_aversion=0.2, liquidity=1.5, sigma=0.1)
mm.run()
mm.results()

                   Results              
----------------------------------------
     statistic                 value
----------------------------------------
Average spread :                 2.50
        Profit :               279.86
   Std(Profit) :                55.10
 End Inventory :                   -4
    Total Fees :                42.52
